In [1]:
import cv2
import numpy as np
import imutils
import os
from matplotlib import pyplot as plt

In [2]:
cv2.namedWindow("Output", cv2.WINDOW_NORMAL)

def detect_ball(prev_frame, frame):
    global optimal_h
    global optimal_w
    global update_counter
    frame_rgb = cv2.cvtColor(src=frame.copy(), code=cv2.COLOR_BGR2RGB)
    prev_frame_rgb = cv2.cvtColor(src=prev_frame.copy(), code=cv2.COLOR_BGR2RGB)

    frame_gray = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2GRAY)
    prepared_frame_ = cv2.GaussianBlur(src= frame_gray, ksize=(15,15), sigmaX=0)
    
    prev_frame_gray = cv2.cvtColor(prev_frame_rgb, cv2.COLOR_BGR2GRAY)
    prepared_prev_frame = cv2.GaussianBlur(src= prev_frame_gray, ksize=(15,15), sigmaX=0)
    
    # calculate difference and update previous frame
    diff_frame = cv2.absdiff(src1=prepared_prev_frame, src2=prepared_frame)

    kernel = np.ones((5,5))
    diff_frame = cv2.erode(diff_frame, kernel, iterations = 10)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
    diff_frame = cv2.dilate(diff_frame_dilated, kernel, iterations = 20)
    
    diff_frame_eroded = cv2.erode(diff_frame_dilated , kernel, iterations = 15)

    thresh_frame = cv2.threshold(src= diff_frame_eroded, thresh=5, maxval=255, type=cv2.THRESH_BINARY)[1]
    
    contours, _ = cv2.findContours(image= thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    #cv2.imshow('output',diff_frame)
    annotated_frame = frame.copy()
    
    number_of_balls = str(len(contours)) if contours else "no balls"
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (100,100)
    fontScale              = 4
    fontColor              = (255, 77, 77)
    thickness              = 4
    lineType               = 2

    cv2.putText(annotated_frame,f"Number of balls: {number_of_balls}", 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType)
     if not contours: return False, annotated_frame, []
    points = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if w < 0.9*optimal_w or h < 0.9*optimal_w: return False, annotated_frame, []
        optimal_w = int((optimal_w * update_counter + w)/(update_counter+1))
        optimal_h = int((optimal_h * update_counter + h)/(update_counter+1))
        cv2.rectangle(annotated_frame,(x,y),(x+optimal_w,y+optimal_h),(0,255,0),5)
        update_counter += 1
        points.append((x,y))
    return True,annotated_frame, points

optimal_w, optimal_h = 0,0
update_counter = 0
i = 0
ret, prev_frame = cap.read()
points_f = []
first_found = False

kf = cv2.KalmanFilter(4, 2)
kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

def Estimate(X, Y):
    global kf
    measured = np.array([[np.float32(X)], [np.float32(Y)]])
    kf.correct(measured)
    predicted = kf.predict()
    return predicted
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
        
    ### Detect the ball ###
    found, annotated_frame,points = detect_ball(prev_frame, frame)
    if found: 
        points_f.append(points[0])
        print("Kalman correct")
        first_found = True
        pred = Estimate(points[0][0],points[0][1])
        cv2.rectangle(annotated_frame,(int(pred[0][0]),int(pred[1][0])),(int(pred[0][0])+optimal_w,int(pred[1][0])+optimal_h),(255,0,0),5)
    if not found and first_found:
        pred = Estimate(points_f[-1][0],points_f[-1][1])
        print(f"Prev: {points_f[-1]}")
        print(f"Pred: {(pred[0][0],pred[1][0])}")
        points_f.append((pred[0][0],pred[1][0]))

        cv2.rectangle(annotated_frame,(int(pred[0][0]),int(pred[1][0])),(int(pred[0][0])+optimal_w,int(pred[1][0])+optimal_h),(255,0,0),5)
        
    ### If the ball is found, update the Kalman filter ###
    
    ### Predict the next state
    
    ### Draw the current tracked state and the predicted state on the image frame ###
    prev_frame = frame
    # Show the frame
    cv2.imshow('output', annotated_frame)
    cv2.waitKey(50)
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
img_path = "C:/Users/subbu/Final_Project/seq_03/img_02/data"
files = os.listdir(img_path)

for file in files:
    if file.endswith(".png"):
        img_l = cv2.imread(os.path.join(img_path, file))

        cv2.imshow("Image", img)
        cv2.waitKey(0)